In [ ]:
# Alteration attack
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

seed = 10000
np.random.seed(seed)
attack_proportion = [0.2, 0.4, 0.6, 0.8, 1.0]

dataset = "covertype"
loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
watermarked_data = loaded_results['watermarked_data']

cover_types = watermarked_data['Cover_Type'].unique()
cover_types.sort()

np.random.seed(111)
for proportion in attack_proportion:
        temp = watermarked_data.copy()
        indices = np.random.choice(len(temp), size=int(proportion * len(temp)), replace=False)
        perturb_values = np.random.choice(cover_types, size=int(proportion * len(temp)))
        temp.loc[indices, 'Cover_Type'] = perturb_values
        temp.to_csv(f"alteration_{dataset}-{seed}-{proportion}.csv", index=False)


In [ ]:
# detect alteration
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import xgboost as xgb
import numpy as np
import math

dataset = 'covertype'
seed = 10000

gamma = 1/2 # ratio between the length of green domain and red domain


# 读取数据
origin = pd.read_csv("../dataset/covtype_with_key.subset.data")

proportions = [0.2, 0.4, 0.6, 0.8, 1.0]

for proportion in proportions:
    loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
    watermarked_data = pd.read_csv(f"alteration_{dataset}-{seed}-{proportion}.csv")

    cover_types = watermarked_data['Cover_Type'].unique()
    cover_types.sort()  

    green_cell = 0
    n_cell = 0
    
    for idx in range(len(watermarked_data)):
        np.random.seed(seed)

        
        selected_data = watermarked_data.loc[idx, columns_of_interest]
        first_two_digits_data = selected_data.apply(first_two_digits)
        composite_numbers = ''.join(first_two_digits_data.values)
        
        if(hash_mod(composite_numbers, 10) != 0):
            continue
        
        n_cell += 1
        
        candidate_set = cover_types

        shuffled_cover_types = list(cover_types)
        np.random.shuffle(shuffled_cover_types)

        half_size = len(shuffled_cover_types) // 2
        green_domain = shuffled_cover_types[:half_size]
        red_domain = shuffled_cover_types[half_size:]

        if watermarked_data.loc[idx, 'Cover_Type'] in green_domain:
            green_cell += 1
    
    # print(n_cell)
    z_score = (green_cell - n_cell/2) / math.sqrt(n_cell/4)
    print(f"{proportion}: The z-score is ", z_score)



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import xgboost as xgb
import numpy as np
import math

dataset = 'covertype'
seed = 10000

gamma = 1/2 # ratio between the length of green domain and red domain


# 读取数据
origin = pd.read_csv("../dataset/covtype_with_key.subset.data")

proportions = [0.2, 0.4, 0.6, 0.8, 1.0]

for proportion in proportions:
    loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
    watermarked_data = pd.read_csv(f"alteration_{dataset}-{seed}-{proportion}.csv")

    cover_types = watermarked_data['Cover_Type'].unique()
    cover_types.sort()  

    X = watermarked_data.drop(columns=['Cover_Type'])
    y = watermarked_data['Cover_Type']

    le = LabelEncoder()
    y = le.fit_transform(y)

    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.3, random_state=42)

    X = origin.drop(columns=['Cover_Type'])
    y = origin['Cover_Type']

    le = LabelEncoder()
    y = le.fit_transform(y)

    _, X_test, _, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = xgb.XGBClassifier(n_estimators=30, max_depth=10, n_jobs=4)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    f1_scores = f1_score(y_test, y_pred, average=None)
    print(f"{proportion}:")
    for i, score in enumerate(f1_scores):
        print(f"Category {le.inverse_transform([i])[0]}: F1-score = {score:.4f}")




In [ ]:
# Insertion attack
import os
import numpy as np
import pandas as pd

seed = 10000
np.random.seed(seed)
attack_proportion = [0.2, 0.4, 0.6, 0.8, 1.0]

dataset = "covertype"
loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
watermarked_data = loaded_results['watermarked_data']

def insert_tuples(temp, proportion):
    num_insertions = int(proportion * len(temp))
    sampled_rows = pd.DataFrame(columns=temp.columns)
    
    for column in temp.columns:
        sampled_rows[column] = temp[column].sample(n=num_insertions, replace=True).reset_index(drop=True)
    
    insertion_indices = np.random.choice(len(temp) + num_insertions, size=num_insertions, replace=False)
    expanded_temp = pd.DataFrame(index=range(len(temp) + num_insertions), columns=temp.columns)
    original_indices = np.setdiff1d(np.arange(len(temp) + num_insertions), insertion_indices)
    
    expanded_temp.iloc[original_indices] = temp.values
    expanded_temp.iloc[insertion_indices] = sampled_rows.values
    
    return expanded_temp


for proportion in attack_proportion:
    temp = watermarked_data.copy()
    temp = insert_tuples(temp, proportion)
    
    temp.to_csv(f"insertion_{dataset}-{seed}-{proportion}.csv", index=False)


In [ ]:
# detect insertion
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import xgboost as xgb
import numpy as np
import math

dataset = 'covertype'
seed = 10000

gamma = 1/2 # ratio between the length of green domain and red domain


# 读取数据
origin = pd.read_csv("../dataset/covtype_with_key.subset.data")

proportions = [0.2, 0.4, 0.6, 0.8, 1.0]

for proportion in proportions:
    loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
    watermarked_data = pd.read_csv(f"insertion_{dataset}-{seed}-{proportion}.csv")

    cover_types = watermarked_data['Cover_Type'].unique()
    cover_types.sort()  

    green_cell = 0
    n_cell = 0
    
    for idx in range(len(watermarked_data)):
        np.random.seed(seed)

        
        selected_data = watermarked_data.loc[idx, columns_of_interest]
        first_two_digits_data = selected_data.apply(first_two_digits)
        composite_numbers = ''.join(first_two_digits_data.values)
        
        if(hash_mod(composite_numbers, 10) != 0):
            continue
        
        n_cell += 1
        
        candidate_set = cover_types

        shuffled_cover_types = list(cover_types)
        np.random.shuffle(shuffled_cover_types)

        half_size = len(shuffled_cover_types) // 2
        green_domain = shuffled_cover_types[:half_size]
        red_domain = shuffled_cover_types[half_size:]

        if watermarked_data.loc[idx, 'Cover_Type'] in green_domain:
            green_cell += 1
    
    # print(n_cell)
    z_score = (green_cell - n_cell/2) / math.sqrt(n_cell/4)
    print(f"{proportion}: The z-score is ", z_score)



In [ ]:
# Deletion attack
import os
import torch
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

seed = 10000
np.random.seed(seed)
attack_proportion = [0.2, 0.4, 0.6, 0.8, 1.0]

dataset = "covertype"
loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
watermarked_data = loaded_results['watermarked_data']

for proportion in attack_proportion:
    temp = watermarked_data.copy()
    indices = np.random.choice(len(temp), size=int(proportion * len(temp)), replace=False)
    temp = temp.drop(indices)
    temp.to_csv(f"deletion_{dataset}-{seed}-{proportion}.csv", index=False)


In [ ]:
# detect insertion
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import xgboost as xgb
import numpy as np
import math

dataset = 'covertype'
seed = 10000

gamma = 1/2 # ratio between the length of green domain and red domain


# 读取数据
origin = pd.read_csv("../dataset/covtype_with_key.subset.data")

proportions = [0.2, 0.4, 0.6, 0.8]

for proportion in proportions:
    loaded_results = np.load(f"{dataset}-{seed}.npy", allow_pickle=True).item()
    watermarked_data = pd.read_csv(f"deletion_{dataset}-{seed}-{proportion}.csv")

    cover_types = watermarked_data['Cover_Type'].unique()
    cover_types.sort()  

    green_cell = 0
    n_cell = 0
    
    for idx in range(len(watermarked_data)):
        np.random.seed(seed)

        
        selected_data = watermarked_data.loc[idx, columns_of_interest]
        first_two_digits_data = selected_data.apply(first_two_digits)
        composite_numbers = ''.join(first_two_digits_data.values)
        
        if(hash_mod(composite_numbers, 10) != 0):
            continue
        
        n_cell += 1
        
        candidate_set = cover_types

        shuffled_cover_types = list(cover_types)
        np.random.shuffle(shuffled_cover_types)

        half_size = len(shuffled_cover_types) // 2
        green_domain = shuffled_cover_types[:half_size]
        red_domain = shuffled_cover_types[half_size:]

        if watermarked_data.loc[idx, 'Cover_Type'] in green_domain:
            green_cell += 1
    
    # print(n_cell)
    z_score = (green_cell - n_cell/2) / math.sqrt(n_cell/4)
    print(f"{proportion}: The z-score is ", z_score)

